Version 1.0.1

# Check your versions

In [ ]:
#import sys
#!conda install --yes --prefix {sys.prefix} lightgbm

In [ ]:
import numpy as np
import pandas as pd 
import sklearn
import scipy.sparse 
import lightgbm 

for p in [np, pd, scipy, sklearn, lightgbm]:
    print (p.__name__, p.__version__)

**Important!** There is a huge chance that the assignment will be impossible to pass if the versions of `lighgbm` and `scikit-learn` are wrong. The versions being tested:

    numpy 1.13.1
    pandas 0.20.3
    scipy 0.19.1
    sklearn 0.19.0
    ligthgbm 2.0.6
    

To install an older version of `lighgbm` you may use the following command:
```
pip uninstall lightgbm
pip install lightgbm==2.0.6
```

# Ensembling

In this programming assignment you are asked to implement two ensembling schemes: simple linear mix and stacking.

We will spend several cells to load data and create feature matrix, you can scroll down this part or try to understand what's happening.

In [1]:
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
%matplotlib inline 

pd.set_option('display.max_rows', 600)
pd.set_option('display.max_columns', 50)

import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from tqdm import tqdm_notebook

from itertools import product


def downcast_dtypes(df):
    '''
        Changes column types in the dataframe: 
                
                `float64` type to `float32`
                `int64`   type to `int32`
    '''
    
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    # Downcast
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int32)
    
    return df

# Load data subset

Let's load the data from the hard drive first.

In [2]:
sales = pd.read_csv('../original_data/sales_train.csv.gz')
shops = pd.read_csv('../original_data/shops.csv')
items = pd.read_csv('../original_data/items.csv')
item_cats = pd.read_csv('../original_data/item_categories.csv')

And use only 3 shops for simplicity.

In [3]:
#mess with this for a sec by commenting
sales = sales[sales['shop_id'].isin([26, 27, 28])]

# Get a feature matrix

We now need to prepare the features. This part is all implemented for you.

In [4]:
# Create "grid" with columns
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = []
mysum=0
for block_num in sales['date_block_num'].unique():
    cur_shops = sales.loc[sales['date_block_num'] == block_num, 'shop_id'].unique()
    cur_items = sales.loc[sales['date_block_num'] == block_num, 'item_id'].unique()
    #print('len(cur_shops)='+str(len(cur_shops)))
    #print('len(cur_items)='+str(len(cur_items)))
    mysum += len(cur_shops)*len(cur_items)
    
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))
save_grid = grid
# Turn the grid into a dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)
print('grid.shape='+str(grid.shape))
# Groupby data to get shop-item-month aggregates
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target':'sum'}})
# Fix column names
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values] 
# Join it to the grid
all_data = pd.merge(grid, gb, how='left', on=index_cols).fillna(0)

# Same as above but with shop-month aggregates
gb = sales.groupby(['shop_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':{'target_shop':'sum'}})
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
all_data = pd.merge(all_data, gb, how='left', on=['shop_id', 'date_block_num']).fillna(0)

# Same as above but with item-month aggregates
gb = sales.groupby(['item_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':{'target_item':'sum'}})
gb.columns = [col[0] if col[-1] == '' else col[-1] for col in gb.columns.values]
all_data = pd.merge(all_data, gb, how='left', on=['item_id', 'date_block_num']).fillna(0)

# Downcast dtypes from 64 to 32 bit to save memory
del grid, gb 
print('There should be '+str(mysum)+' many rows')
gc.collect();

grid.shape=(278619, 3)


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\groupby\groupby.py:4656: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


There should be 278619 many rows


In [ ]:
save_grid[0]

In [ ]:
np.mean(all_data.target==0)

In [ ]:
all_data[all_data.date_block_num==0].shape

In [ ]:
all_data.shape

In [5]:
all_data.head()

,shop_id,item_id,date_block_num,target,target_shop,target_item
0,28,7738,0,4.0,7057.0,11.0
1,28,7737,0,10.0,7057.0,16.0
2,28,7770,0,6.0,7057.0,10.0
3,28,7664,0,1.0,7057.0,1.0
4,28,7814,0,2.0,7057.0,6.0


In [6]:
all_data[(all_data.date_block_num==3) & (all_data.shop_id==28)]

,shop_id,item_id,date_block_num,target,target_shop,target_item
32454,28,3903,3,1.0,4910.0,1.0
32455,28,3738,3,7.0,4910.0,14.0
32456,28,3739,3,1.0,4910.0,3.0
32457,28,3743,3,1.0,4910.0,5.0
32458,28,3619,3,2.0,4910.0,3.0
32459,28,3651,3,1.0,4910.0,1.0
32460,28,3656,3,3.0,4910.0,5.0
32461,28,3661,3,1.0,4910.0,1.0
32462,28,3834,3,2.0,4910.0,4.0
32463,28,3768,3,1.0,4910.0,1.0


In [7]:
all_data[(all_data.date_block_num==33) & (all_data.shop_id==28)]

,shop_id,item_id,date_block_num,target,target_shop,target_item
275265,28,3352,33,21.0,3018.0,29.0
275266,28,3353,33,13.0,3018.0,15.0
275267,28,3479,33,2.0,3018.0,2.0
275268,28,3480,33,1.0,3018.0,3.0
275269,28,3372,33,1.0,3018.0,1.0
275270,28,3350,33,18.0,3018.0,22.0
275271,28,3351,33,35.0,3018.0,46.0
275272,28,3385,33,1.0,3018.0,1.0
275273,28,3546,33,1.0,3018.0,1.0
275274,28,3552,33,2.0,3018.0,2.0


In [8]:
all_data.shape

(278619, 6)

After creating a grid, we can calculate some features. We will use lags from [1, 2, 3, 4, 5, 12] months ago.

In [9]:
# List of columns that we will use to create lags
cols_to_rename = list(all_data.columns.difference(index_cols)) 

shift_range = [1, 2, 3, 4, 5, 12]

for month_shift in shift_range:
    train_shift = all_data[index_cols + cols_to_rename].copy()
    
    train_shift['date_block_num'] = train_shift['date_block_num'] + month_shift
    
    foo = lambda x: '{}_lag_{}'.format(x, month_shift) if x in cols_to_rename else x
    train_shift = train_shift.rename(columns=foo)

    all_data = pd.merge(all_data, train_shift, on=index_cols, how='left').fillna(0)

del train_shift

# Don't use old data from year 2013
all_data = all_data[all_data['date_block_num'] >= 12] 

# List of all lagged features
fit_cols = [col for col in all_data.columns if col[-1] in [str(item) for item in shift_range]] 
# We will drop these at fitting stage
to_drop_cols = list(set(list(all_data.columns)) - (set(fit_cols)|set(index_cols))) + ['date_block_num'] 

# Category for each item
item_category_mapping = items[['item_id','item_category_id']].drop_duplicates()

all_data = pd.merge(all_data, item_category_mapping, how='left', on='item_id')
all_data = downcast_dtypes(all_data)
gc.collect();

In [10]:
all_data.shape

(157950, 25)

In [11]:
all_data

,shop_id,item_id,date_block_num,target,target_shop,target_item,target_lag_1,target_item_lag_1,target_shop_lag_1,target_lag_2,target_item_lag_2,target_shop_lag_2,target_lag_3,target_item_lag_3,target_shop_lag_3,target_lag_4,target_item_lag_4,target_shop_lag_4,target_lag_5,target_item_lag_5,target_shop_lag_5,target_lag_12,target_item_lag_12,target_shop_lag_12,item_category_id
0,28,10994,12,1.0,6949.0,1.0,0.0,1.0,8499.0,0.0,1.0,6454.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,37
1,28,10992,12,3.0,6949.0,4.0,3.0,7.0,8499.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7521.0,0.0,0.0,0.0,37
2,28,10991,12,1.0,6949.0,5.0,1.0,3.0,8499.0,0.0,0.0,0.0,0.0,1.0,5609.0,0.0,2.0,6753.0,2.0,4.0,7521.0,0.0,0.0,0.0,40
3,28,10988,12,1.0,6949.0,2.0,2.0,5.0,8499.0,4.0,5.0,6454.0,5.0,6.0,5609.0,0.0,2.0,6753.0,0.0,0.0,0.0,0.0,0.0,0.0,40
4,28,11002,12,1.0,6949.0,1.0,0.0,1.0,8499.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40
5,28,10987,12,1.0,6949.0,1.0,0.0,0.0,0.0,1.0,1.0,6454.0,2.0,2.0,5609.0,5.0,5.0,6753.0,0.0,0.0,0.0,0.0,0.0,0.0,37
6,28,11339,12,2.0,6949.0,3.0,1.0,3.0,8499.0,0.0,0.0,0.0,1.0,1.0,5609.0,1.0,2.0,6753.0,1.0,2.0,7521.0,0.0,0.0,0.0,43
7,28,10970,12,2.0,6949.0,2.0,0.0,1.0,8499.0,1.0,1.0,6454.0,0.0,0.0,0.0,3.0,5.0,6753.0,2.0,2.0,7521.0,0.0,0.0,0.0,40
8,28,10969,12,1.0,6949.0,2.0,0.0,0.0,0.0,0.0,1.0,6454.0,0.0,0.0,0.0,0.0,1.0,6753.0,0.0,0.0,0.0,0.0,0.0,0.0,40
9,28,10968,12,2.0,6949.0,3.0,1.0,1.0,8499.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40


To this end, we've created a feature matrix. It is stored in `all_data` variable. Take a look:

In [ ]:
#sales.groupby(['shop_id','date_block_num']).item_cnt_day.sum()

In [ ]:
b = (all_data['shop_id']==28) & (all_data['item_id']==10994)
all_data[b]

In [ ]:
all_data.shape

# Train/test split

For a sake of the programming assignment, let's artificially split the data into train and test. We will treat last month data as the test set.

In [ ]:
# Save `date_block_num`, as we can't use them as features, but will need them to split the dataset into parts 
dates = all_data['date_block_num']

last_block = dates.max()
print('Test `date_block_num` is %d' % last_block)

In [ ]:
dates_train = dates[dates <  last_block]
dates_test  = dates[dates == last_block]

X_train = all_data.loc[dates <  last_block].drop(to_drop_cols, axis=1)
X_test =  all_data.loc[dates == last_block].drop(to_drop_cols, axis=1)

y_train = all_data.loc[dates <  last_block, 'target'].values
y_test =  all_data.loc[dates == last_block, 'target'].values

In [ ]:
X_train

# First level models 

You need to implement a basic stacking scheme. We have a time component here, so we will use ***scheme f)*** from the reading material. Recall, that we always use first level models to build two datasets: test meta-features and 2-nd level train-metafetures. Let's see how we get test meta-features first. 

### Test meta-features

Firts, we will run *linear regression* on numeric columns and get predictions for the last month.

In [ ]:
lr = LinearRegression()
lr.fit(X_train.values, y_train)
pred_lr = lr.predict(X_test.values)

print('Test R-squared for linreg is %f' % r2_score(y_test, pred_lr))

And the we run *LightGBM*.

In [ ]:
lgb_params = {
               'feature_fraction': 0.75,
               'metric': 'rmse',
               'nthread':1, 
               'min_data_in_leaf': 2**7, 
               'bagging_fraction': 0.75, 
               'learning_rate': 0.03, 
               'objective': 'mse', 
               'bagging_seed': 2**7, 
               'num_leaves': 2**7,
               'bagging_freq':1,
               'verbose':0 
              }

model = lgb.train(lgb_params, lgb.Dataset(X_train, label=y_train), 100)
pred_lgb = model.predict(X_test)

print('Test R-squared for LightGBM is %f' % r2_score(y_test, pred_lgb))

Finally, concatenate test predictions to get test meta-features.

In [ ]:
X_test_level2 = np.c_[pred_lr, pred_lgb] 

In [ ]:
np.c_

### Train meta-features

**Now it is your turn to write the code**. You need to implement ***scheme f)*** from the reading material. Here, we will use duration **T** equal to month and **M=15**.  

That is, you need to get predictions (meta-features) from *linear regression* and *LightGBM* for months 27, 28, 29, 30, 31, 32. Use the same parameters as in above models.

In [ ]:
dates_train_level2 = dates_train[dates_train.isin([27, 28, 29, 30, 31, 32])]

# That is how we get target for the 2nd level dataset
y_train_level2 = y_train[dates_train.isin([27, 28, 29, 30, 31, 32])]

In [ ]:
y_train

In [ ]:
dates_train_level2

In [ ]:
type(dates_train)

In [ ]:
# And here we create 2nd level feeature matrix, init it with zeros first
X_train_level2 = np.zeros([y_train_level2.shape[0], 2])
X_train_level2

In [ ]:
X_train.head()

In [ ]:
#print(dates_train.shape)
#X_train_meta = X_train[dates_train<cur_block_num]
#X_train_meta
#X_train[X_train['date_block_num']]

In [ ]:
# Now fill `X_train_level2` with metafeatures
for cur_block_num in [27, 28, 29, 30, 31, 32]:
    
    print(cur_block_num)
    
    '''
        1. Split `X_train` into parts
           Remember, that corresponding dates are stored in `dates_train` 
        2. Fit linear regression 
        3. Fit LightGBM and put predictions          
        4. Store predictions from 2. and 3. in the right place of `X_train_level2`. 
           You can use `dates_train_level2` for it
           Make sure the order of the meta-features is the same as in `X_test_level2`
    '''      
    X_train_meta = X_train[dates_train<cur_block_num]
    Y_train_meta = y_train[dates_train<cur_block_num]
    X_to_predict_on = X_train[dates_train==cur_block_num]
    lr.fit(X_train_meta,Y_train_meta)
    lr_pred = lr.predict(X_to_predict_on)
    
    
    model = lgb.train(lgb_params,lgb.Dataset(X_train_meta,label=Y_train_meta),100)
    lgb_pred = model.predict(X_to_predict_on)
    
    #print(lr_pred.shape)
    #print(lgb_pred.shape)
    #print(X_train_level2[dates_train_level2==cur_block_num].shape)
    X_train_level2[dates_train_level2==cur_block_num] = np.c_[lr_pred,lgb_pred]
    #  YOUR CODE GOES HERE
    
    
# Sanity check
assert np.all(np.isclose(X_train_level2.mean(axis=0), [ 1.50148988,  1.38811989]))

In [ ]:
X_train_level2.mean(axis=0)
#y_train.mean(axis=0)

Remember, the ensembles work best, when first level models are diverse. We can qualitatively analyze the diversity by examinig *scatter plot* between the two metafeatures. Plot the scatter plot below. 

In [ ]:
pd.DataFrame(X_train_level2)
pd.DataFrame(X_train_level2).plot.scatter(x=0,y=1)

In [ ]:
X_train_level2

# Ensembling

Now, when the meta-features are created, we can ensemble our first level models.

### Simple convex mix

Let's start with simple linear convex mix:

$$
mix= \alpha\cdot\text{linreg_prediction}+(1-\alpha)\cdot\text{lgb_prediction}
$$

We need to find an optimal $\alpha$. And it is very easy, as it is feasible to do grid search. Next, find the optimal $\alpha$ out of `alphas_to_try` array. Remember, that you need to use train meta-features (not test) when searching for $\alpha$. 

In [ ]:
alphas_to_try = np.linspace(0, 1, 1001)
best_alpha=-1
best_r2=-10000000000000000
for alpha in alphas_to_try:
    mix = alpha*X_train_level2[:,0] + (1-alpha)*X_train_level2[:,1]
    r2=r2_score(y_train_level2,mix)
    if r2 > best_r2:
        best_alpha=alpha
        best_r2=r2
    print(alpha)
    print(r2)
    print('\n')
print('best alpha=',best_alpha)
print('best_r2=',best_r2)
r2_train_simple_mix = best_r2
    

In [ ]:
best_alpha

In [ ]:
#most of this is unnecesary, I way overthought it, theres no clever validation here, just perfectly fit alpha
alphas_to_try = np.linspace(0, 1, 1001)
best_alpha=-1
best_r2=-10000000000000000
for alpha in alphas_to_try:
            print('alpha=',alpha)
            meta_model_pred = alpha*X_train_level2[:,0] + (1-alpha)*X_train_level2[:,1]

            r2 = r2_score(meta_model_pred,y_train_level2)
            print('r2=',r2,'\n')
            if r2 > best_r2:
                best_r2 = r2
                best_alpha=alpha
            #new_row = pd.DataFrame(data=[[alpha,r2]],columns=['alpha','r2'])
            #cv_df = cv_df.append(new_row)
            #
            

# YOUR CODE GOES HERE
#best_alpha = # YOUR CODE GOES HERE
print('best_alpha=',best_alpha)
r2_train_simple_mix = best_r2

In [ ]:
print(dates_train_level2.unique())
print(X_train_level2.shape)

In [ ]:
X_train[dates > 26].shape

In [ ]:
alphas_to_try = np.linspace(0, 2, 10)#1001)

#use months 27 - 29 for 1st fold
#then 27 - 30
#then 27 - 31
validation_months = [30,31,32]
cv_df = pd.DataFrame(columns=['alpha','r2'])
cv_df

In [ ]:
#pd.DataFrame(data=[[1,2,3]],columns=['alpha','month','r2'])

In [ ]:
#cv_df = cv_df.append(pd.DataFrame(data=[[1,2,3]],columns=['alpha','month','r2']))
#cv_df.loc[-1]=[1,2,4]
cv_df

In [ ]:
#meta_train[:,1]

In [ ]:
#most of this is unnecesary, I way overthought it, theres no clever validation here, just perfectly fit alpha
#best_alpha=-1
#best_r2=-10000000000000000
#for alpha in alphas_to_try:
#            print('alpha=',alpha)
        
#            meta_train = X_train_level2[dates_train_level2 < validation_month]
#            meta_target = y_train_level2[dates_train_level2 < validation_month]
            
            #fit a metamodel to meta_train ~ meta_target
#            lr.fit(meta_train,meta_target)
#            model = lgb.train(lgb_params,lgb.Dataset(meta_train,label=meta_target),100)
            #
            
#            meta_valid_X = X_train_level2[dates_train_level2 == validation_month]
#            meta_valid_Y = y_train_level2[dates_train_level2 == validation_month]
            #make predictions for validation month
            #lr_pred = lr.predict(meta_valid_X)
            #lgb_pred = model.predict(meta_valid_X)
#            meta_model_pred = alpha*X_train_level2[:,0] + (1-alpha)*X_train_level2[:,1]

            #print(alpha)
            #print(validation_month)
            #print(r2_score(meta_model_pred,y_train_level2))
#            r2 = r2_score(meta_model_pred,y_train_level2)
#            new_row = pd.DataFrame(data=[[alpha,validation_month,r2]],columns=['alpha','month','r2'])
#            cv_df = cv_df.append(new_row)
            
#            if r2 > best_r2:
#                best_r2 = r2
#                best_alpha=alpha
            #
            

# YOUR CODE GOES HERE
#best_alpha = # YOUR CODE GOES HERE
#r2_train_simple_mix = best_r2

In [ ]:
r2_train_simple_mix = best_r2

In [ ]:
best_alpha

In [ ]:
cv_df
#bad performance in block 32

In [ ]:
cv_df.groupby('alpha').r2.mean()

In [ ]:
#best_alpha=1
#r2_train_simple_mix =0.485940 

In [ ]:
print('Best alpha: %f; Corresponding r2 score on train: %f' % (best_alpha, r2_train_simple_mix))

Now use the $\alpha$ you've found to compute predictions for the test set 

In [ ]:
test_preds = best_alpha*X_test_level2[:,0] + (1-best_alpha)*X_test_level2[:,1]
r2_test_simple_mix = r2_score(y_test,test_preds)

print('Test R-squared for simple mix is %f' % r2_test_simple_mix)

In [ ]:
y_test

### Stacking

Now, we will try a more advanced ensembling technique. Fit a linear regression model to the meta-features. Use the same parameters as in the model above.

In [ ]:
print(X_train_level2.shape)
print(y_train_level2.shape)

In [ ]:
lr.fit(X_train_level2,y_train_level2)
lr.predict(X_test_level2)

In [ ]:
            validation_month=32        
            meta_train = X_train_level2[dates_train_level2 < validation_month]
            meta_target = y_train_level2[dates_train_level2 < validation_month]
            
            #fit a metamodel to meta_train ~ meta_target
            lr.fit(meta_train,meta_target)
            #
            
            meta_valid_X = X_train_level2[dates_train_level2 == validation_month]
            meta_valid_Y = y_train_level2[dates_train_level2 == validation_month]

Compute R-squared on the train and test sets.

In [ ]:
lr.fit(X_train_level2,y_train_level2)


train_preds = lr.predict(X_train_level2)
r2_train_stacking = r2_score(y_train_level2,train_preds)

test_preds = lr.predict(X_test_level2)
r2_test_stacking = r2_score(y_test,test_preds)

print('Train R-squared for stacking is %f' % r2_train_stacking)
print('Test  R-squared for stacking is %f' % r2_test_stacking)

Interesting, that the score turned out to be lower than in previous method. Although the model is very simple (just 3 parameters) and, in fact, mixes predictions linearly, it looks like it managed to overfit. **Examine and compare** train and test scores for the two methods. 

And of course this particular case does not mean simple mix is always better than stacking.

We all done! Submit everything we need to the grader now.

In [ ]:
#r2_test_simple_mix=0
#r2_train_stacking=0
#r2_test_stacking=0

In [ ]:
from grader import Grader
grader = Grader()

grader.submit_tag('best_alpha', best_alpha)

grader.submit_tag('r2_train_simple_mix', r2_train_simple_mix)
grader.submit_tag('r2_test_simple_mix',  r2_test_simple_mix)

grader.submit_tag('r2_train_stacking', r2_train_stacking)
grader.submit_tag('r2_test_stacking',  r2_test_stacking)

In [ ]:
STUDENT_EMAIL = 'jordan.james.sands@gmail.com'
STUDENT_TOKEN = 'N8c4wgkEtshGQLPc'
grader.status()

In [ ]:
grader.submit(STUDENT_EMAIL, STUDENT_TOKEN)